In [168]:
import requests
import util.item_analysis as ui
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
# 예
market = ['심리 상담 센터']
addr = '서울시 영등포구 여의도동'

# 고객이 입력한 상품을 파는 매장을 리스트로 가져오기
market_list = market

In [169]:
# 고객이 입력한 주소 ['시', '구'] 또는 ['군', '면', '리'] 가져오기
addr_list = addr.split(' ')
addr_list1 = []
addr_list2 = []

for i in addr_list:
    if i[-1] == '시' or i[-1] == '구' or i[-1] == '동':
        addr_list1.append(i)
    elif i[-1] == '군' or i[-1] == '면' or i[-1] == '리':
        addr_list2.append(i)

addr1 = ' '.join(addr_list1)
addr2 = ' '.join(addr_list2)

if addr1 != '':
    addr_result = addr1
else:
    addr_result = addr2

addr_result

'서울시 영등포구 여의도동'

In [144]:
# # 고객이 입력한 주소 좌표 구하기
# try:
#     with open('keys/카카오api.txt') as file:
#         kakao_key = file.read()

#     base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
#     header = {'Authorization': f'KakaoAK {kakao_key}'}
#     url = f'{base_url}?query={quote(addr)}'
#     result = requests.get(url, headers=header).json()
#     lat = float(result['documents'][0]['y'])
#     lng = float(result['documents'][0]['x'])
# except:
#     print('주소 형식이 올바르지 않습니다.')

In [170]:
lat, lng

(37.5263635705036, 126.896273923703)

In [171]:
# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)

In [172]:
# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

In [173]:
# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)

In [174]:
# '상세 주소 화살표' 클릭
try:
    juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
    juso_way[0].click()
    time.sleep(1)
except:
    print('근처에 매장이 없습니다')

In [175]:
# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source)
market_list = soup.find('ul', class_='eDFz9')
mart = market_list.find('li', recursive=False)
mart

<li class="VLTHu" data-laim-exp-id="undefined"><div class="qbGlu"><div class="TTfa9"><a class="place_thumb" href="https://m.place.naver.com/place/1281149859/photo?entry=ple" role="button"><div class="lazyload-wrapper"><img alt="같이심리상담센터" data-grid-lazy="false" height="88" src="https://search.pstatic.net/common/?autoRotate=true&amp;quality=95&amp;type=f160_160&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20201230_62%2F1609313148858HX6e1_JPEG%2FxRdJHXlbZciVIIgYMBtACzf0.jpg" width="88"/></div><span class="place_thumb_count"><span class="place_blind">이미지수</span>8</span></a></div><div class="ouxiq"><a class="P7gyV" href="https://m.place.naver.com/place/1281149859?entry=ple" role="button"><div class="ApCpt"><div class="place_bluelink C6RjW"><span class="YwYLL">같이심리상담센터</span><span class="YzBgS">심리상담</span></div></div></a><a class="P7gyV" href="https://m.place.naver.com/place/1281149859?entry=ple" role="button"><div class="iHXbN"><div class="Gvf9B"><span class="XP3ml yTY83">영업 중</span><span 

In [195]:
center_list.select_one('.JsCty > a').get('href')[4:]

'02-2679-1911'

In [176]:
market_list.select_one('div.AbTyi').get_text().split('복사')[1][2:]

'서울특별시 영등포구 당산동1가 306 백양빌딩 501호'

In [197]:
results = []

# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)

# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)



# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source, 'html.parser')
center_list = soup.find('ul', class_='eDFz9')

for i in range(5):
    if center_list:
        center = center_list.select('li', recursive=False)[i]

        if center:
            # 현재 위치에서 마켓 거리
            c_distance_element = center.select_one('span.lWwyx.NVngW')
            c_distance = c_distance_element.get_text().split('서')[1] if c_distance_element else 'N/A'

            # 마켓 이름
            if center.select_one('span.YwYLL'):
                c_title = center.select_one('span.YwYLL').get_text() 
            else:
                c_title = 'N/A'   

            # '상세 주소 화살표' 클릭
            try:
                juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
                juso_way[i].click()
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                center_list = soup.find('ul', class_='eDFz9')
                # 마켓 도로명 주소
                if center_list.select_one('div.zZfO1'):
                    if center_list.select_one('div.zZfO1').get_text()[:2] == '도로':
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[0][3:]
                    elif center_list.select_one('div.zZfO1').get_text()[:2] == '지번':
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[1][2:]
                    else:
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[1][2:]
                else:
                    c_addr = 'N/A' 
                
                # 전화번호 추출
                juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
                juso_way[i].click()
                c_phone_element = center_list.select_one('.JsCty > a')
                c_phone = c_phone_element.get('href')[4:] if c_phone_element else 'N/A'
                
                time.sleep(1)
            except:
                print('근처에 매장이 없습니다')
                
            # 딕셔너리 형태로 저장
            center_data = {
                '거리': c_distance,
                '매장명': c_title,
                '주소': c_addr,
                '전화번호': c_phone
            }
            # 딕셔너리 형태로 저장한 것을 리스트에 저장 (3가지 마켓을 넣어야 하므로)
            results.append(center_data)

        else:
            print('마켓 정보를 찾을 수 없습니다.')
    else:
        print('마켓 리스트를 찾을 수 없습니다.')

results_df = pd.DataFrame(results)
print(results_df)


AttributeError: 'NoneType' object has no attribute 'click'

In [180]:
center_list.select_one('div.gkDbw')

In [153]:
df = pd.DataFrame(results)
df

,거리,매장명,주소,전화번호
0,220m,닥터앤맘,서울특별시 영등포구 양산로17길 23 1층,N/A
1,300m,같이심리상담센터,서울특별시 영등포구 양산로 128 백양빌딩 501호,N/A
2,350m,당신의 마음거울,서울특별시 영등포구 당산로18길 3-4 1층,N/A
3,380m,다빈치심리상담센터,서울특별시 영등포구 양산로 142,N/A
4,530m,서울예술심리상담센터,서울특별시 영등포구 당산로41길 11 당산skV1센터 E동 B123호,N/A
...,...,...,...,...
96,4.1km,마인드줌심리상담센터,서울특별시 마포구 월드컵로 190 이안상암 2차 2층 210호,N/A
97,4.2km,착한벗심리상담센터,서울특별시 동작구 여의대방로20길 61 슬기동 2층,N/A
98,4.3km,동작구청소년상담복지센터,서울특별시 마포구 양화로 175,N/A
99,4.3km,마인드페이지,서울특별시 동작구 성대로2길 9 5층,N/A


In [154]:
# 고객이 입력한 주소 좌표 구하기
from tqdm import tqdm
with open('keys/카카오api.txt') as file:
    kakao_key = file.read()

base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}
lat_list, lng_list = [], []

for i in tqdm(df.index):
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))


  0%|          | 0/101 [00:00<?, ?it/s]

100%|██████████| 101/101 [00:06<00:00, 15.48it/s]


In [155]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,거리,매장명,주소,전화번호,위도,경도
0,220m,닥터앤맘,서울특별시 영등포구 양산로17길 23 1층,N/A,37.526683,126.893757
1,300m,같이심리상담센터,서울특별시 영등포구 양산로 128 백양빌딩 501호,N/A,37.523786,126.897201
2,350m,당신의 마음거울,서울특별시 영등포구 당산로18길 3-4 1층,N/A,37.523204,126.896655
3,380m,다빈치심리상담센터,서울특별시 영등포구 양산로 142,N/A,37.523503,126.898800
4,530m,서울예술심리상담센터,서울특별시 영등포구 당산로41길 11 당산skV1센터 E동 B123호,N/A,37.530811,126.898319
...,...,...,...,...,...,...
96,4.1km,마인드줌심리상담센터,서울특별시 마포구 월드컵로 190 이안상암 2차 2층 210호,N/A,37.564178,126.903290
97,4.2km,착한벗심리상담센터,서울특별시 동작구 여의대방로20길 61 슬기동 2층,N/A,37.493869,126.921806
98,4.3km,동작구청소년상담복지센터,서울특별시 마포구 양화로 175,N/A,37.557754,126.924365
99,4.3km,마인드페이지,서울특별시 동작구 성대로2길 9 5층,N/A,37.499244,126.931549


In [156]:
df.to_csv('data/상담센터.csv')


In [157]:
df = pd.read_csv('data/상담센터.csv')
df.head()

,Unnamed: 0,거리,매장명,주소,전화번호,위도,경도
0,0,220m,닥터앤맘,서울특별시 영등포구 양산로17길 23 1층,NaN,37.526683,126.893757
1,1,300m,같이심리상담센터,서울특별시 영등포구 양산로 128 백양빌딩 501호,NaN,37.523786,126.897201
2,2,350m,당신의 마음거울,서울특별시 영등포구 당산로18길 3-4 1층,NaN,37.523204,126.896655
3,3,380m,다빈치심리상담센터,서울특별시 영등포구 양산로 142,NaN,37.523503,126.898800
4,4,530m,서울예술심리상담센터,서울특별시 영등포구 당산로41길 11 당산skV1센터 E동 B123호,NaN,37.530811,126.898319


In [158]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df

,거리,매장명,주소,전화번호,위도,경도
0,220m,닥터앤맘,서울특별시 영등포구 양산로17길 23 1층,NaN,37.526683,126.893757
1,300m,같이심리상담센터,서울특별시 영등포구 양산로 128 백양빌딩 501호,NaN,37.523786,126.897201
2,350m,당신의 마음거울,서울특별시 영등포구 당산로18길 3-4 1층,NaN,37.523204,126.896655
3,380m,다빈치심리상담센터,서울특별시 영등포구 양산로 142,NaN,37.523503,126.898800
4,530m,서울예술심리상담센터,서울특별시 영등포구 당산로41길 11 당산skV1센터 E동 B123호,NaN,37.530811,126.898319
...,...,...,...,...,...,...
96,4.1km,마인드줌심리상담센터,서울특별시 마포구 월드컵로 190 이안상암 2차 2층 210호,NaN,37.564178,126.903290
97,4.2km,착한벗심리상담센터,서울특별시 동작구 여의대방로20길 61 슬기동 2층,NaN,37.493869,126.921806
98,4.3km,동작구청소년상담복지센터,서울특별시 마포구 양화로 175,NaN,37.557754,126.924365
99,4.3km,마인드페이지,서울특별시 동작구 성대로2길 9 5층,NaN,37.499244,126.931549


In [159]:
df = df.astype({'위도':'float'})
df

,거리,매장명,주소,전화번호,위도,경도
0,220m,닥터앤맘,서울특별시 영등포구 양산로17길 23 1층,NaN,37.526683,126.893757
1,300m,같이심리상담센터,서울특별시 영등포구 양산로 128 백양빌딩 501호,NaN,37.523786,126.897201
2,350m,당신의 마음거울,서울특별시 영등포구 당산로18길 3-4 1층,NaN,37.523204,126.896655
3,380m,다빈치심리상담센터,서울특별시 영등포구 양산로 142,NaN,37.523503,126.898800
4,530m,서울예술심리상담센터,서울특별시 영등포구 당산로41길 11 당산skV1센터 E동 B123호,NaN,37.530811,126.898319
...,...,...,...,...,...,...
96,4.1km,마인드줌심리상담센터,서울특별시 마포구 월드컵로 190 이안상암 2차 2층 210호,NaN,37.564178,126.903290
97,4.2km,착한벗심리상담센터,서울특별시 동작구 여의대방로20길 61 슬기동 2층,NaN,37.493869,126.921806
98,4.3km,동작구청소년상담복지센터,서울특별시 마포구 양화로 175,NaN,37.557754,126.924365
99,4.3km,마인드페이지,서울특별시 동작구 성대로2길 9 5층,NaN,37.499244,126.931549


In [160]:
import folium
map = folium.Map([df.위도.mean(), df.경도.mean()], zoom_start=13)
map

In [161]:
map = folium.Map([df.위도.mean(), df.경도.mean()], zoom_start=14)
for i in df.index:
    folium.Marker([df.위도[i], df.경도[i]], tooltip= [df.매장명[i], '<br>', df.주소[i]]).add_to(map)

map

In [162]:
def map_by_dong(df, dong):
    gf = df[df.BJDONG_NM.str.find(dong) == 0.0] 
    map = folium.Map([gf.위도.mean(), gf.경도.mean()], zoom_start=15)
    for i in gf.index:
        folium.Marker([df.위도[i], df.경도[i]], tooltip= [df.매장명[i], '<br>', df.주소[i]]).add_to(map)

    map.save('dong.html')

    map